# Treasury Gold dataset Exploration


Ideas for graphs:
~~* Simple stacked column chart of amount of gold by location over time~~
  * Same things but option for by book value
~~* Pie Chart of form desc~~
* fIndicators
  * highest amount at any one location
  * Source line num filter
* Bubble chart
  * another version or upgrade to one on a map

In [1]:
# Importing packages
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import panel as pn
import json
from owen_helper import monthConv
pn.extension()

In [2]:
# Fetch data from the API
api_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v2/accounting/od/gold_reserve" # Get first 60000 rows
# Parameters to include in the API call
params = {'page[size]': 1500}

# Make the API call with the specified parameters
response = requests.get(api_url, params=params)
data = response.json()

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data['data'])

df['location_desc'] = df['location_desc'].replace("Federal Reserve Banks - NY Vault", "FRB NY Vault")
df['location_desc'] = df['location_desc'].replace("Federal Reserve Banks - Display", "FRB Display")
df['location_desc'] = df['location_desc'].replace("All Locations- Coins, blanks, miscellaneous", "All Locations- Miscellaneous")
df['fine_troy_ounce_qty_mill'] = pd.to_numeric(df['fine_troy_ounce_qty'], errors='coerce')/1000000


df.head()

,record_date,facility_desc,form_desc,location_desc,fine_troy_ounce_qty,book_value_amt,src_line_nbr,record_fiscal_year,record_fiscal_quarter,record_calendar_year,record_calendar_quarter,record_calendar_month,record_calendar_day,fine_troy_ounce_qty_mill
0,2012-01-31,Mint Held Gold - Deep Storage,Gold Bullion,"Denver, CO",43853707.279,1851599995.81,1,2012,2,2012,1,01,31,43.853707
1,2012-01-31,Mint Held Gold - Deep Storage,Gold Bullion,"Fort Knox, KY",147341858.382,6221097412.78,2,2012,2,2012,1,01,31,147.341858
2,2012-01-31,Mint Held Gold - Deep Storage,Gold Bullion,"West Point, NY",54067331.379,2282841677.17,3,2012,2,2012,1,01,31,54.067331
3,2012-01-31,Mint Held Gold - Working Stock,Gold Coins,All Locations- Miscellaneous,2783218.656,117513614.74,4,2012,2,2012,1,01,31,2.783219
4,2012-01-31,Federal Reserve Bank Held Gold,Gold Bullion,FRB NY Vault,13376961.126,564804728.53,5,2012,2,2012,1,01,31,13.376961


## Chart Ideas

The below are simple chart ideas for this dataset that are developed here just to see usefulness and then can be applied to BI or Panel later

In [3]:
# choose a random year, in future this will be passed into function on panel/BI
selected_year = 2013
selected_month = 10

In [4]:
min_year = int(df['record_calendar_year'].min())
max_year = int(df['record_calendar_year'].max())
widget_css={
    'background': '#f8f8f8',
    'border': '1px solid black',
    'padding': '10px',
    # 'box-shadow': '5px 5px 5px #bcbcbc'
}
month_widget_css={
    'background': '#f8f8f8',
    'border': '1px solid black',
    'padding': '20px',
    # 'box-shadow': '5px 5px 5px #bcbcbc'
}
year_select_widget = pn.widgets.IntInput(name='Input Year', value=min_year, step=1, start=min_year, end=max_year,styles=month_widget_css)

month_select_widget = pn.widgets.Select(name='Input Month', options=["January", "February", "March", "April","May","June","July","August","September","October","November",'December'],styles=widget_css)
type_select_widget = pn.widgets.Select(name='Data Type', options=["Value", "Weight"],styles=widget_css)
pie_type_select_widget = pn.widgets.Select(name='Data Type', options=["Form", "Location"],styles=widget_css)


In [5]:
def gold_comp_pie(input_year,input_month_name,type_select):
    month_num_as_str =str(monthConv(input_month_name))
    
    if len(str(month_num_as_str)) == 1:
        month_num_as_str = "0"+month_num_as_str
    month_year_filtered_df = df.loc[(df['record_calendar_year'] == str(input_year)) & (df['record_calendar_month'] == month_num_as_str)]

    # Group the data by 'form_desc' and calculate the sum of 'fine_troy_ounce_qty'
    if type_select == "Form":
        grouped_df = month_year_filtered_df.groupby('form_desc')['fine_troy_ounce_qty_mill'].sum()
        labeldist = 1.05
        pctdist = .8
    else:
        grouped_df = month_year_filtered_df.groupby('location_desc')['fine_troy_ounce_qty_mill'].sum()
        labeldist = 1.2
        pctdist = 1.1
    # Plotting the pie chart

    fig, ax = plt.subplots(figsize=(8,8))
    grouped_df.plot(kind='pie', autopct='%1.1f%%',pctdistance=pctdist,labeldistance=labeldist)

    # Set the title and labels for the chart
    plt.title(f'Gold Form in Storage Type in {input_month_name} {str(input_year)}')
    ax.set_ylabel('')
    # fig.set_facecolor('black')
    # ax.xaxis.label.set_color('white')
    # ax.yaxis.label.set_color('white')


    # Hide the chart
    plt.close()
    return pn.pane.Matplotlib(fig)

In [6]:
pie_multi_dash = pn.Column(
    "# U.S. Treasury-Owned Gold",
    pn.Row(year_select_widget,month_select_widget,pie_type_select_widget),
    pn.bind(gold_comp_pie,input_year=year_select_widget,input_month_name=month_select_widget,type_select=pie_type_select_widget)
)
# Set the background color using custom CSS

pie_multi_dash

BokehModel(combine_events=True, render_bundle={'docs_json': {'3354a5bf-2895-41ec-8b2e-e461c1921204': {'version…

In [7]:
def stacked_bar(input_year,type_select):
    # Convert 'fine_troy_ounce_qty' column to numeric type
    df['fine_troy_ounce_qty_mill'] = pd.to_numeric(df['fine_troy_ounce_qty'], errors='coerce')/1000000
    df['book_value_amt_mill'] = pd.to_numeric(df['book_value_amt'], errors='coerce')/1000000

    # Filter DataFrame for the desired year
    year_filtered_df = df.loc[df['record_calendar_year'] == str(input_year)]
    fig, ax = plt.subplots(figsize=(10,9))
    # Group the data by location and record_calendar_year and calculate the sum of fine_troy_ounce_qty
    if type_select == "Value":
        grouped_df = year_filtered_df.groupby(['record_calendar_month','record_calendar_year','location_desc' ])['book_value_amt_mill'].sum().unstack()
        ax.set_ylabel('Millions of Dollars')
        ax.set_title(f'Value of Gold by Location Over Time for {input_year}')
    else:
        grouped_df = year_filtered_df.groupby(['record_calendar_month','record_calendar_year','location_desc' ])['fine_troy_ounce_qty_mill'].sum().unstack()
        ax.set_ylabel('Million Fine Troy Ounces')
        ax.set_title(f'Weight of Gold by Location Over Time for {input_year}')


    # Plotting the stacked column chart

    # Plotting the stacked column chart
    grouped_df.plot(kind='bar', stacked=True, ax=ax)

    # Set the title and labels for the chart
    ax.set_xlabel('Record Calendar Year')
    fig.set_facecolor('#fff')

    # Show the chart
    plt.close()
    return pn.pane.Matplotlib(fig)

In [8]:
stacked_Bar = pn.Column(
    "# U.S. Treasury-Owned Gold",
    pn.Row(year_select_widget,type_select_widget),
    pn.bind(stacked_bar,input_year=year_select_widget,type_select=type_select_widget)
)
# Set the background color using custom CSS

stacked_Bar

BokehModel(combine_events=True, render_bundle={'docs_json': {'5b84eb2a-a046-464f-83d2-cbce7c554492': {'version…

In [9]:
#nasdaq api key 3cFs55zpDsrPpzgSwDTi
nasdaq_data = requests.get("https://data.nasdaq.com/api/v3/datasets/LBMA/GOLD.json?api_key=3cFs55zpDsrPpzgSwDTi")
data_json = json.loads(nasdaq_data.text)
json_formatted_str = json.dumps(data_json, indent=2)

print(json_formatted_str)

{
  "dataset": {
    "id": 11304240,
    "dataset_code": "GOLD",
    "database_code": "LBMA",
    "name": "Gold Price: London Fixing",
    "description": "Gold Price: London Fixings, London Bullion Market Association (LBMA). Fixing levels are set per troy ounce. The London Gold Fixing Companies set the prices for gold that are globally considered as the international standard for pricing of gold. The Gold price in London is set twice a day by five LBMA Market Makers who comprise the London Gold Market Fixing Limited (LGMFL). The process starts with the announcement from the Chairman of the LGMFL to the other members of the LBMA Market Makers, then relayed to the dealing rooms where customers can express their interest as buyers or sellers and also the quantity they wish to trade. The gold fixing price is then set by collating bids and offers until the supply and demand are matched. At this point the price is announced as the 'Fixed' price for gold and all business is conducted on the b

In [10]:
real_gold_df = pd.DataFrame(data_json['dataset']['data'])
real_gold_df.columns = ["Date", "USD (AM)","USD (PM)","GBP (AM)","GBP (PM)","EUR (AM)", "EUR (PM)"]
real_gold_df

,Date,USD (AM),USD (PM),GBP (AM),GBP (PM),EUR (AM),EUR (PM)
0,2023-06-26,1930.55,1922.90,1516.060,1512.17,1771.86,1762.15
1,2023-06-23,1919.35,1930.70,1510.530,1519.27,1768.30,1773.54
2,2023-06-22,1927.95,1920.05,1509.010,1505.51,1753.08,1748.59
3,2023-06-21,1935.25,1925.65,1521.110,1514.33,1771.14,1762.74
4,2023-06-20,1953.85,1930.45,1528.110,1516.75,1787.38,1767.91
...,...,...,...,...,...,...,...
14015,1968-01-08,35.14,NaN,14.586,NaN,NaN,NaN
14016,1968-01-05,35.14,NaN,14.597,NaN,NaN,NaN
14017,1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN
14018,1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN


In [11]:

# add row to df that is amount of gold times price, then u can use groupby and sum
def get_real_value(input_year, input_month_name):
    month_num_as_str =str(monthConv(input_month_name))
    if len(str(month_num_as_str)) == 1:
        month_num_as_str = "0"+month_num_as_str
    # if year-mm in date
    month_year_filtered_real_gold_df = real_gold_df.loc[real_gold_df["Date"].str.startswith(f"{input_year}-{month_num_as_str}")]
    price_av = sum(month_year_filtered_real_gold_df["USD (AM)"])/len(month_year_filtered_real_gold_df["USD (AM)"])
    return price_av


get_real_value(2022,"March")

1947.417391304348